## 导入相关的包

In [112]:
import pandas as pd
import numpy as np


## 读取数据集

In [73]:
ft_zodiac = pd.read_csv('data/ft_zodiac.txt')
zodiac_label = pd.read_csv('data/zodiac_label.txt')

查看数据集信息

In [74]:
print(ft_zodiac.shape)
ft_zodiac.head()

(23519, 4)


,0,order_id,chinese_zodiac,zodiac
0,0,100000081567592448,狗,处女座
1,0,100000467565182976,牛,双子座
2,0,100000530945323008,羊,射手座
3,0,100000556765458432,鼠,摩羯座
4,0,100000598171623424,虎,水瓶座


In [75]:
len(set(ft_zodiac.zodiac))#查看星座是否为12个全

12

## 定义好人和坏人的标签

pd15作为好坏的分割节点。>15 为坏人，<15为好人？ 

15天以上的人为坏，标签为１；5天以内的人为好标签为０；５至１５天的标签为２。

In [76]:
ft_label=zodiac_label[zodiac_label['label']!=2]#此处是吧5-15天的人去掉，因为邻近时间的人具有相似性，所以选择”挖个洞“这种处理方式
ft_label.head()

,order_id,overdue_days,repay_time,label
0,100000081567592448,0,2018-07-09,0
1,100000467565182976,1,2018-07-09,0
2,100000530945323008,0,2018-07-09,0
3,100000556765458432,0,2018-07-09,0
4,100000598171623424,0,2018-07-09,0


In [77]:
set(zodiac_label.label)

{0, 1, 2}

In [78]:
data=pd.merge(ft_label,ft_zodiac,on= 'order_id',how='inner')#拼接两张表
data.head()

,order_id,overdue_days,repay_time,label,0,chinese_zodiac,zodiac
0,100000081567592448,0,2018-07-09,0,0,狗,处女座
1,100000467565182976,1,2018-07-09,0,0,牛,双子座
2,100000530945323008,0,2018-07-09,0,0,羊,射手座
3,100000556765458432,0,2018-07-09,0,0,鼠,摩羯座
4,100000598171623424,0,2018-07-09,0,0,虎,水瓶座


badrate=bad/total

## 星座坏账率分析

In [79]:
zodiac_list=set(data.zodiac)

In [80]:
#星座
zodiac_badrate = {}
for x in zodiac_list:
    a=data[data.zodiac==x]
    bad = a[a.label==1]['label'].count()
    good =  a[a.label==0]['label'].count()
    
    zodiac_badrate[x]=bad/(bad+good)

In [81]:
zodiac_badrate

{'白羊座': 0.13455414012738853,
 '天秤座': 0.12461252324860508,
 '水瓶座': 0.140117994100295,
 '双子座': 0.1312410841654779,
 '处女座': 0.13035143769968052,
 '摩羯座': 0.12920489296636087,
 '天蝎座': 0.12005028284098052,
 '双鱼座': 0.14873646209386282,
 '狮子座': 0.12760416666666666,
 '巨蟹座': 0.1408351026185421,
 '金牛座': 0.12259059367771781,
 '射手座': 0.14480286738351256}

In [82]:
f = zip(zodiac_badrate.keys(),zodiac_badrate.values())
f = sorted(f,key= lambda x:x[1],reverse=True)
zodiac_badrate=pd.DataFrame(f)
zodiac_badrate.columns = pd.Series(['星座','badrate'])
zodiac_badrate.head()

,星座,badrate
0,双鱼座,0.148736
1,射手座,0.144803
2,巨蟹座,0.140835
3,水瓶座,0.140118
4,白羊座,0.134554


In [116]:
from pyecharts import options as opts
from pyecharts.charts import Line
x = list(zodiac_badrate['星座'])
y = list(zodiac_badrate['badrate'])

line = (
    Line()
    .add_xaxis(x)
    .add_yaxis("Bad Rate", y, is_smooth=True)
    .set_global_opts(title_opts=opts.TitleOpts(title="星座 Bad Rate 折线图"))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
line.render_notebook()

差距太小，所以星座没有太多的参考价值

## 生肖坏账率分析

In [119]:
chinese_zodiac_list=set(data.chinese_zodiac)

In [122]:
#生肖
chinese_zodiac_badrate = {}
for x in chinese_zodiac_list:
    a=data[data.chinese_zodiac==x]
    bad = a[a.label==1]['label'].count()
    good =  a[a.label==0]['label'].count()
    
    chinese_zodiac_badrate[x]=bad/(bad+good)

In [123]:
chinese_zodiac_badrate

{'羊': 0.11335403726708075,
 '猴': 0.12008141112618724,
 '蛇': 0.12789827973074047,
 '马': 0.12053872053872054,
 '龙': 0.1439084219133279,
 '兔': 0.1502843216896832,
 '猪': 0.14269406392694065,
 '虎': 0.15165876777251186,
 '鸡': 0.12846998063266624,
 '鼠': 0.1340602950609365,
 '狗': 0.11052009456264776,
 '牛': 0.1578112609040444}

In [124]:
f = zip(chinese_zodiac_badrate.keys(),chinese_zodiac_badrate.values())
f = sorted(f,key= lambda x:x[1],reverse=True)
chinese_zodiac_badrate=pd.DataFrame(f)
chinese_zodiac_badrate.columns = pd.Series(['生肖','badrate'])
chinese_zodiac_badrate.head()

,生肖,badrate
0,牛,0.157811
1,虎,0.151659
2,兔,0.150284
3,龙,0.143908
4,猪,0.142694


In [125]:
x = list(chinese_zodiac_badrate['生肖'])
y = list(chinese_zodiac_badrate['badrate'])

line = (
    Line()
    .add_xaxis(x)
    .add_yaxis("Bad Rate", y, is_smooth=True)
    .set_global_opts(title_opts=opts.TitleOpts(title="生肖 Bad Rate 折线图"))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
line.render_notebook()